In [2]:
import pandas as pd
import numpy as np

In [3]:
# Assume tar.gz file to be extracted at project directory.
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
train_batch_1 = unpickle('cifar-10-batches-py/data_batch_1')
train_batch_2 = unpickle('cifar-10-batches-py/data_batch_2')
train_batch_3 = unpickle('cifar-10-batches-py/data_batch_3')
train_batch_4 = unpickle('cifar-10-batches-py/data_batch_4')
train_batch_5 = unpickle('cifar-10-batches-py/data_batch_5')
test_batch = unpickle('cifar-10-batches-py/test_batch')

In [5]:
def reshape_image(data):
    data = np.reshape(data, (3,32,32))
    data = np.moveaxis(data, 0, -1)

    return data

In [6]:
def filter_classes(batch, label):
    x = []
    y = []

    true_count = 0
    false_count = 0

    labels = np.array([int(i == label) for i in batch[b'labels']])

    _, counts = np.unique(labels, return_counts=True)
    n = min(counts)

    for i in range(len(batch[b'data'])):
        if batch[b'labels'][i] == label:
            if true_count < n:
                x.append(reshape_image(batch[b'data'][i]))
                y.append(1)
                true_count += 1
        else:
            if false_count < n:
                x.append(reshape_image(batch[b'data'][i]))
                y.append(0)
                false_count += 1

    return np.array(x), np.array(y)

In [7]:
train_x1, train_y1 = filter_classes(train_batch_1, 0)
train_x2, train_y2 = filter_classes(train_batch_2, 0)
train_x3, train_y3 = filter_classes(train_batch_3, 0)
train_x4, train_y4 = filter_classes(train_batch_4, 0)
train_x5, train_y5 = filter_classes(train_batch_5, 0)

train_x = np.concatenate((train_x1, train_x2, train_x3, train_x4, train_x5))
train_y = np.concatenate((train_y1, train_y2, train_y3, train_y4, train_y5))
test_x, test_y = filter_classes(test_batch, 0)

In [8]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow as tf
from tensorflow.keras import Input, layers, Sequential, optimizers, losses, callbacks

In [9]:
HP_NUM_UNITS_1 = hp.HParam('num_units_1', hp.Discrete([28, 32]))
HP_NUM_UNITS_2 = hp.HParam('num_units_2', hp.Discrete([48, 52]))
HP_NUM_UNITS_3 = hp.HParam('num_units_3', hp.Discrete([68, 72]))
HP_NUM_UNITS_4 = hp.HParam('num_units_4', hp.Discrete([100, 104]))

HP_DROPOUT_3 = hp.HParam('dropout_3', hp.Discrete([0.2]))

METRIC_ACCURACY = 'accuracy'



In [10]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[
            HP_NUM_UNITS_1,
            HP_NUM_UNITS_2,
            HP_NUM_UNITS_3,
            HP_NUM_UNITS_4,
            HP_DROPOUT_3,
            ],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [11]:
def train_test_model(hparams):
    data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal",
        input_shape=(32,32,3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
    )

    model = Sequential(name="Image_Recognition_Model")
    model.add(Input(shape=(32,32,3,))) # Input layer.

    model.add(data_augmentation)
    model.add(layers.Rescaling(scale=1./255, name="Normaliser")) # Example pre-processing layer.

    model.add(layers.Conv2D(hparams[HP_NUM_UNITS_1], 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(hparams[HP_NUM_UNITS_2], 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(hparams[HP_NUM_UNITS_3], 3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(hparams[HP_DROPOUT_3]))

    model.add(layers.Flatten())
    model.add(layers.Dense(hparams[HP_NUM_UNITS_4], activation='relu'))
    model.add(layers.Dense(2))
    

    model.compile(
        optimizer='adam', #optimizers.SGD(learning_rate=0.1),
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    model.fit(
        train_x,
        train_y,
        epochs=5,
        # callbacks=[
        #     callbacks.TensorBoard('output\\logs'),
        #     hp.KerasCallback('output\\logs', hparams),
        # ]
        verbose=0,
    )

    _, accuracy = model.evaluate(test_x, test_y)

    return accuracy

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [13]:
session_num = 0

for num_units_1 in HP_NUM_UNITS_1.domain.values:
    for num_units_2 in HP_NUM_UNITS_2.domain.values:
        for num_units_3 in HP_NUM_UNITS_3.domain.values:
            for num_units_4 in HP_NUM_UNITS_4.domain.values:
                for dropout_3 in HP_DROPOUT_3.domain.values:
                    hparams = {
                        HP_NUM_UNITS_1: num_units_1,
                        HP_NUM_UNITS_2: num_units_2,
                        HP_NUM_UNITS_3: num_units_3,
                        HP_NUM_UNITS_4: num_units_4,
                        HP_DROPOUT_3: dropout_3,
                    }
                    run_name = "run-%d" % session_num
                    print('--- Starting trial: %s' % run_name)
                    print({h.name: hparams[h] for h in hparams})
                    run('logs\\hparam_tuning\\' + run_name, hparams)
                    session_num += 1


--- Starting trial: run-0
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 68, 'num_units_4': 100, 'dropout_3': 0.2}
63/63 [==============================] - 1s 9ms/step - loss: 0.3862 - accuracy: 0.8335
--- Starting trial: run-1
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 68, 'num_units_4': 104, 'dropout_3': 0.2}
63/63 [==============================] - 1s 7ms/step - loss: 0.3473 - accuracy: 0.8450
--- Starting trial: run-2
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 72, 'num_units_4': 100, 'dropout_3': 0.2}
63/63 [==============================] - 1s 9ms/step - loss: 0.3734 - accuracy: 0.8460
--- Starting trial: run-3
{'num_units_1': 28, 'num_units_2': 48, 'num_units_3': 72, 'num_units_4': 104, 'dropout_3': 0.2}
63/63 [==============================] - 1s 9ms/step - loss: 0.4185 - accuracy: 0.8055
--- Starting trial: run-4
{'num_units_1': 28, 'num_units_2': 52, 'num_units_3': 68, 'num_units_4': 100, 'dropout_3': 0.2}
63/63 [==============================] - 1

In [15]:
%reload_ext tensorboard
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 28024), started 17:49:03 ago. (Use '!kill 28024' to kill it.)